In [1]:
# Code to pull hash data
import requests
import csv
from datetime import datetime, timedelta

# Function to fetch data for a specific day and append it to the CSV file
def fetch_data_and_append_to_csv(date, csv_writer):
    date_milliseconds = int(date.timestamp()) * 1000
    url = f"https://blockchain.info/blocks/{date_milliseconds}?format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for block in data:
            csv_writer.writerow([block["hash"], block["height"], datetime.utcfromtimestamp(block["time"]).strftime('%Y-%m-%d %H:%M:%S'), block["block_index"]])
    else:
        print(f"Failed to retrieve data for {date}. Status Code: {response.status_code}")

# Get today's date
end_date = datetime.now()

# Calculate the start date (7 days ago)
start_date = end_date - timedelta(days=6)

# Create a filename with the date range
date_range_string = start_date.strftime("%Y-%m-%d") + "_to_" + end_date.strftime("%Y-%m-%d")
filename = f"blockchain_data.csv"

# Open the file in write mode
with open(filename, "w", newline='') as file:
    csv_writer = csv.writer(file)
    # Write header
    csv_writer.writerow(["hash", "height", "time", "block_index"])
    # Fetch and append data for each day in the last 7 days
    current_date = start_date
    while current_date <= end_date:
        fetch_data_and_append_to_csv(current_date, csv_writer)
        current_date += timedelta(days=1)

print(f"Data for the last 7 days has been written to {filename}.")

Data for the last 7 days has been written to blockchain_data.csv.


In [2]:
# Code to pull single blocks
import requests
import csv

# List of JSON keys
json_keys = ["hash", "ver", "prev_block", "mrkl_root", "time", "bits", "next_block", "fee", "nonce", "n_tx", "size", "block_index", "main_chain", "height", "weight"]

# Function to fetch block data for a given hash and append it to the CSV file
def fetch_block_data_and_append_to_csv(block_hash, csv_writer):
    try:
        url = f"https://blockchain.info/rawblock/{block_hash}"
        response = requests.get(url)

        print(f"Data pulled for {block_hash} successfully!")

        if response.status_code == 200:
            data = response.json()

            if 'tx' in data:
                del data['tx']  # Skip the 'tx' field
                
            csv_writer.writerow([
                data.get("hash", ""),
                data.get("ver", ""),
                data.get("prev_block", ""),
                data.get("mrkl_root", ""),
                data.get("time", ""),
                data.get("bits", ""),
                data.get("next_block", ""),
                data.get("fee", ""),
                data.get("nonce", ""),
                data.get("n_tx", ""),
                data.get("size", ""),
                data.get("block_index", ""),
                data.get("main_chain", ""),
                data.get("height", ""),
                data.get("weight", "")
            ])
        else:
            print(f"Request for {block_hash} failed with status code {response.status_code}.")

    except requests.exceptions.RequestException as e:
        print(f"Request for {block_hash} failed with an exception: {e}")

    except ValueError as e:
        print(f"Value error occurred while processing the response for {block_hash}: {e}")

# Read block data hash from the CSV file
with open("blockchain_data.csv", "r") as file:
    csv_reader = csv.reader(file)
    hash_values = [row[0] for row in csv_reader]

# Write block data to a separate CSV file
with open("blockchain_block_data.csv", "w", newline='') as file:
    csv_writer = csv.writer(file)
    firstwrite = True
    for i, block_hash in enumerate(hash_values):
        
        #Logic to add header to the file
        if firstwrite:
            csv_writer.writerow(json_keys)
            firstwrite = False
            print(json_keys, " appended!")

        fetch_block_data_and_append_to_csv(block_hash, csv_writer)

print("Block data appended to blockchain_block_data.csv successfully.")

['hash', 'ver', 'prev_block', 'mrkl_root', 'time', 'bits', 'next_block', 'fee', 'nonce', 'n_tx', 'size', 'block_index', 'main_chain', 'height', 'weight']  appended!
Data pulled for hash successfully!
Request for hash failed with status code 404.
Data pulled for 000000000000000000035929380d2da0feb22be900b4aded585f9759c925ca61 successfully!
Data pulled for 00000000000000000000e457f8637044051772b0726da0b90929500db243cbee successfully!
Data pulled for 000000000000000000028e741345faf050d125344dad6eef9722f197c01f24f1 successfully!
Data pulled for 000000000000000000007487e0a145a8c911faedc8db0d0b940b2448ff4b9822 successfully!
Data pulled for 0000000000000000000282287e3e65608d618691ccecfa801cf4db61a3953a57 successfully!
Data pulled for 0000000000000000000351ae64aaadd2a4f604e3febe58ed370a4966370887b3 successfully!
Data pulled for 000000000000000000028cbe2bf14171712a0c77c0bc2102bd4029e6c4a00e7f successfully!
Data pulled for 00000000000000000000bc30c5905aee9dbec91b7011408255a600bc73c7f494 successf

In [3]:
# Code to change date format
import csv
from datetime import datetime

# Update the CSV file with formatted timestamp
with open('blockchain_block_data.csv', 'r', newline='') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)
    header = rows[0]
    updated_rows = [header]
    for row in rows[1:]:
        timestamp = int(row[4])
        formatted_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        row[4] = formatted_time
        updated_rows.append(row)

with open('blockchain_block_data.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerows(updated_rows)

print("Timestamps in the CSV file have been updated successfully.")

Timestamps in the CSV file have been updated successfully.
